In [101]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [102]:
import numpy as np
import pandas as pd
from math import ceil
import os
import random
import matplotlib.pylab as plt
import plotly.express as px
from zipfile import ZipFile
from PIL import Image, ImageEnhance; Image.MAX_IMAGE_PIXELS = 5_000_000_000

from sklearn.decomposition import PCA

import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras import Sequential, Model
from tensorflow.keras.layers import Dense,MaxPooling2D, MaxPool2D, Conv2D,Flatten,Dropout, Activation, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img

from kaggle_datasets import KaggleDatasets

In [103]:
os.path.abspath(".")

In [104]:
import random


list_of_categories = os.listdir('/kaggle/input/imagenetmini-1000/imagenet-mini/train/')

number_of_categories = len(list_of_categories)

# sample_category = list_of_categories[random.choice(range(number_of_categories))]
sample_category = "n02093647" # Dogs

print(sample_category)

sample_photos_list = os.listdir('/kaggle/input/imagenetmini-1000/imagenet-mini/train/{}'.format(sample_category))


In [105]:
# Plotting samples

plt.figure(figsize=(20, 10))

plt.subplot(2, 2, 1)
sample_photo_filename_1 = random.choice(sample_photos_list)
image_1 = load_img('/kaggle/input/imagenetmini-1000/imagenet-mini/train/{}/{}'.format(sample_category, sample_photo_filename_1))
plt.imshow(image_1)

plt.subplot(2, 2, 2)
sample_photo_filename_2 = random.choice(sample_photos_list)
image_2 = load_img('/kaggle/input/imagenetmini-1000/imagenet-mini/train/{}/{}'.format(sample_category, sample_photo_filename_2))
plt.imshow(image_2)

plt.axis('off')

plt.tight_layout()

In [106]:
# Pixel analysis

sample_photo_filename_3 = random.choice(sample_photos_list)
image_3 = plt.imread('/kaggle/input/imagenetmini-1000/imagenet-mini/train/{}/{}'.format(sample_category, sample_photo_filename_3))

plt.imshow(image_3, cmap='gray')
plt.colorbar()
# plt.title('Dog Image')

print("Width: {} pixels, Height: {} pixels".format(image_3.shape[0], image_3.shape[1]))
print("Max pixel value: {}, Min pixel value: {}".format(image_3.max(), image_3.min()))
print("Mean pixel value: {}, Standard Deviation: {}".format(image_3.mean(), image_3.std()))

In [107]:
# Pixel distribution

import seaborn as sns

sns.distplot(image_3.ravel(),
             label="Pixel Mean {} & Standard Deviation {}".format(np.mean(image_3),np.std(image_3)), 
             kde=False)
plt.legend(loc='upper right')
plt.title('Distribution of Pixel Intensities in the Image')
plt.xlabel('Pixel Intensity')
plt.ylabel('# Pixels in Image')

In [108]:
# Credit: https://towardsdatascience.com/exploratory-data-analysis-ideas-for-image-classification-d3fc6bbfb2d2
def img2np(path, list_of_filename, size = (64, 64)):
    # iterating through each file
    for fn in list_of_filename:
        fp = path + fn
        current_image = image.load_img(fp, target_size = size, 
                                       color_mode = 'grayscale')
        # covert image to a matrix
        img_ts = image.img_to_array(current_image)
        # turn that into a vector / 1D array
        img_ts = [img_ts.ravel()]
        try:
            # concatenate different images
            full_mat = np.concatenate((full_mat, img_ts))
        except UnboundLocalError: 
            # if not assigned yet, assign one
            full_mat = img_ts
    return full_mat

# run it on our folders
dog_images_matrix = img2np('/kaggle/input/imagenetmini-1000/imagenet-mini/train/{}/'.format(sample_category), sample_photos_list)

In [109]:
# Credit: https://towardsdatascience.com/exploratory-data-analysis-ideas-for-image-classification-d3fc6bbfb2d2
def eigenimages(full_mat, title, n_comp = 0.7, size = (64, 64)):
    # fit PCA to describe n_comp * variability in the class
    pca = PCA(n_components = n_comp, whiten = True)
    pca.fit(full_mat)
    print('Number of PC: ', pca.n_components_)
    return pca
  
def plot_pca(pca, size = (64, 64)):
    # plot eigenimages in a grid
    n = pca.n_components_
    fig = plt.figure(figsize=(8, 8))
    r = int(n**.5)
    c = ceil(n/ r)
    for i in range(n):
        ax = fig.add_subplot(r, c, i + 1, xticks = [], yticks = [])
        ax.imshow(pca.components_[i].reshape(size), 
                  cmap='Greys_r')
    plt.axis('off')
    plt.show()

In [110]:
plot_pca(eigenimages(dog_images_matrix, 'Dogs'))

In [111]:
# Building data generators for easier data augmentation. Validation generator has samplewise settings set to False because this information will not be available during test time

train_image_generator = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    samplewise_center=True,
    samplewise_std_normalization=True
)

val_image_generator = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    samplewise_center=False,
    samplewise_std_normalization=False
)

In [112]:
# Setting data sources

train = train_image_generator.flow_from_directory("/kaggle/input/imagenetmini-1000/imagenet-mini/train", 
                                            batch_size=8, 
                                            shuffle=True, 
                                            class_mode='binary',
                                            target_size=(320, 320))

validation = val_image_generator.flow_from_directory("/kaggle/input/imagenetmini-1000/imagenet-mini/val", 
                                                batch_size=1, 
                                                shuffle=False, 
                                                class_mode='binary',
                                                target_size=(320, 320))

In [113]:
# From https://www.kaggle.com/code/dschettler8845/guie-a-better-tensorflow-example-baseline-model

def build_guie_model(wt_path, tf_keras_model_fn):
    _inputs = tf.keras.layers.Input(shape=(None,None,3), batch_size=1, dtype=tf.uint8, name="inputs")
    _bb = tf_keras_model_fn(include_top=False, weights=wt_path)
    x = tf.keras.layers.GlobalAveragePooling2D()(_bb(_inputs))
    x = tf.keras.layers.Reshape((-1,1))(x)
    x = tf.keras.layers.AveragePooling1D(22)(x)
    _output_1 = tf.keras.layers.Reshape((-1,), name="embedding")(x)
    _output_2 = tf.keras.layers.Lambda(lambda x: tf.nn.l2_normalize(x), name="embedding_norm")(_output_1)    
    return tf.keras.Model(inputs=_inputs, outputs=[_output_1, _output_2])

IMAGENET_WEIGHTS = "imagenet"
TF_KERAS_MODEL_FN = tf.keras.applications.EfficientNetB2

guie_model = build_guie_model(IMAGENET_WEIGHTS, TF_KERAS_MODEL_FN)
guie_model.summary()

os.makedirs("/kaggle/models", exist_ok=True)
guie_model.save("/kaggle/models")

In [114]:
from zipfile import ZipFile

with ZipFile('/kaggle/submission.zip','w') as zip:           
  zip.write('/kaggle/models/saved_model.pb', arcname='saved_model.pb') 
  zip.write('/kaggle/models/variables/variables.data-00000-of-00001', arcname='variables/variables.data-00000-of-00001') 
  zip.write('/kaggle/models/variables/variables.index', arcname='variables/variables.index') 

In [115]:
saved_model_path = "/kaggle/models"
image_path       =  "/kaggle/input/imagenetmini-1000/imagenet-mini/val/n03976657/ILSVRC2012_val_00043517.JPEG"

# Model loading.
model = tf.saved_model.load(saved_model_path)
embedding_fn = model.signatures["serving_default"]

# Load image and extract its embedding.
image_tensor = tf.convert_to_tensor(np.array(Image.open(image_path).convert("RGB")))
expanded_tensor = tf.expand_dims(image_tensor, axis=0)
embedding = embedding_fn(expanded_tensor)["embedding_norm"]

print(embedding)